# Preparing Dataset

In [1]:
import cv2
import numpy
import os

import torch
import torch.nn.functional as F

In [5]:
images = []
lbl = []

for folder in os.listdir('Gesture Image Data'):
    for idx, image in enumerate(os.listdir(os.path.join('Gesture Image Data', folder))):
        images.append(cv2.imread(os.path.join('Gesture Image Data', folder, image)))
        lbl.append(folder)

print(len(images),len(lbl))

55500 55500


In [6]:
images[0].shape #pytorch accepts (channel, widht, height)

(50, 50, 3)

In [7]:
for idx, img in enumerate(images):
    images[idx] = numpy.transpose(images[idx])

print(images[100].shape)

(3, 50, 50)


In [8]:
images = torch.FloatTensor(images)

C:\Users\iamza\AppData\Local\Temp\ipykernel_1376\1848181980.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:248.)
  images = torch.FloatTensor(images)


In [9]:
print(images[0])

tensor([[[ 62.,  65.,  63.,  ...,  60.,  58.,  56.],
         [ 58.,  62.,  63.,  ...,  62.,  62.,  60.],
         [ 66.,  67.,  65.,  ...,  60.,  58.,  57.],
         ...,
         [ 93.,  93.,  89.,  ...,  71.,  71.,  67.],
         [ 94.,  96.,  97.,  ...,  78.,  71.,  71.],
         [ 95.,  97., 101.,  ...,  80.,  74.,  72.]],

        [[ 91.,  92.,  89.,  ...,  80.,  79.,  77.],
         [ 85.,  90.,  90.,  ...,  84.,  84.,  83.],
         [ 92.,  90.,  89.,  ...,  82.,  84.,  83.],
         ...,
         [112., 112., 108.,  ...,  90.,  90.,  88.],
         [114., 113., 111.,  ...,  90.,  89.,  89.],
         [115., 115., 112.,  ...,  90.,  90.,  91.]],

        [[128., 129., 125.,  ..., 105., 106., 104.],
         [122., 125., 124.,  ..., 109., 109., 109.],
         [129., 128., 125.,  ..., 107., 108., 107.],
         ...,
         [133., 133., 129.,  ...,  98.,  98.,  96.],
         [132., 132., 130.,  ..., 100.,  96.,  96.],
         [132., 132., 132.,  ..., 100.,  97.,  96.]]]

In [10]:
images = F.normalize(images)
print(images[0])

tensor([[[0.3672, 0.3795, 0.3798,  ..., 0.4138, 0.4018, 0.3972],
         [0.3634, 0.3734, 0.3803,  ..., 0.4108, 0.4108, 0.4012],
         [0.3845, 0.3936, 0.3900,  ..., 0.4066, 0.3903, 0.3880],
         ...,
         [0.4716, 0.4716, 0.4676,  ..., 0.4708, 0.4708, 0.4575],
         [0.4744, 0.4836, 0.4935,  ..., 0.5016, 0.4768, 0.4768],
         [0.4769, 0.4846, 0.5039,  ..., 0.5111, 0.4881, 0.4781]],

        [[0.5390, 0.5372, 0.5365,  ..., 0.5517, 0.5472, 0.5461],
         [0.5326, 0.5420, 0.5433,  ..., 0.5565, 0.5565, 0.5549],
         [0.5360, 0.5287, 0.5341,  ..., 0.5557, 0.5652, 0.5649],
         ...,
         [0.5680, 0.5680, 0.5674,  ..., 0.5968, 0.5968, 0.6009],
         [0.5754, 0.5692, 0.5648,  ..., 0.5787, 0.5976, 0.5976],
         [0.5774, 0.5746, 0.5588,  ..., 0.5750, 0.5936, 0.6042]],

        [[0.7581, 0.7532, 0.7536,  ..., 0.7241, 0.7343, 0.7376],
         [0.7644, 0.7528, 0.7485,  ..., 0.7222, 0.7222, 0.7288],
         [0.7516, 0.7520, 0.7501,  ..., 0.7251, 0.7267, 0.

In [11]:
print(numpy.unique(lbl))

['0' '1' '2' '3' '4' '5' '6' '7' '8' '9' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H'
 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z'
 '_']


In [12]:
lbl = numpy.unique(lbl, return_inverse=True)[1]
print(numpy.unique(lbl))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36]


In [13]:
lbl = torch.LongTensor(lbl)

In [14]:
#if you have gpu
images = images.cuda()
lbl = lbl.cuda()

# Prepare model

In [15]:
import torch.nn as nn
import torch.optim as optim

In [23]:
class conv(nn.Module):
    
    def __init__(self):
        super(conv,self).__init__()
        
        self.conv1 = nn.Conv2d(3, 6 , 5)
        #6@46x46
        #after max_pool 6@23x23
        self.conv2 = nn.Conv2d(6, 12, 5)
        #12@19x19
        #after max_pool 12@9x9
        
        self.fc1 = nn.Linear(9*9*12 , 30)
        self.fc2 = nn.Linear(30 , 37)
        
        
    def forward(self,x):
        
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        x = x.view(-1, 9*9*12)
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [24]:
convnet = conv()
convnet = convnet.cuda()

In [28]:
res = convnet(images[0])

In [30]:
print(res , len(res[0]))

tensor([[ 0.0535, -0.0544,  0.0802, -0.1938, -0.1929,  0.0237, -0.1306,  0.1293,
         -0.1188,  0.1308,  0.1929, -0.0952,  0.0249, -0.0203,  0.0157, -0.1205,
         -0.0909,  0.1684,  0.1306,  0.1977, -0.1584, -0.1681,  0.0421, -0.0161,
          0.1685,  0.0716,  0.0510, -0.0058,  0.0353,  0.1118,  0.1293,  0.0629,
         -0.0147, -0.1361, -0.0363, -0.1714, -0.1788]], device='cuda:0',
       grad_fn=<AddmmBackward0>) 37


# training

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(convnet.parameters(), lr=0.001, momentum=0.9)

In [31]:
from tqdm import tqdm

In [35]:
for epoch in range(1):
    
    running_loss = 0.0
    
    for count in tqdm(range(11100)):
        
        optimizer.zero_grad()
        
        output = convnet(images[count*5 : (count+1)*5])
        loss = criterion(output, lbl[count*5 : (count+1)*5])
        loss.backward()
        optimizer.step()
        
        running_loss+=loss.item()
        
        if count%11099 ==0:
            print('[%d,%5d] loss: %.3f' %
                 (epoch +1, count+1, running_loss/11099))
            
            running_loss=0.0
            
print('done training')

  0%|                                                                               | 12/11100 [00:03<35:12,  5.25it/s]

[1,    1] loss: 0.000


100%|███████████████████████████████████████████████████████████████████████████| 11100/11100 [00:29<00:00, 380.48it/s]

[1,11100] loss: 1.462
done training


In [36]:
torch.save(convnet, 'cnn.pth')

In [37]:
results = convnet(images[0])
print(results)

tensor([[-1.1260, -0.7210, -0.8749, -1.1751, -1.1001, -1.0326, -1.1290, -1.2061,
         -0.9378, -0.9761, -0.9734, -1.0249, -1.0257, -0.9283, -0.8040, -0.7299,
         -0.6089, -0.5065, -0.4132, -0.3496, -0.2621, -0.1874, -0.1067, -0.0161,
          0.0455,  0.1743,  0.2912,  0.4558,  0.5812,  0.7106,  0.9250,  1.1623,
          1.4036,  1.8113,  2.2725,  3.0600,  5.4311]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


In [38]:
model = torch.load('cnn.pth')
model.eval()

conv(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=972, out_features=30, bias=True)
  (fc2): Linear(in_features=30, out_features=37, bias=True)
)

In [39]:
res  = model(images[0])

In [40]:
print(res)

tensor([[-1.1260, -0.7210, -0.8749, -1.1751, -1.1001, -1.0326, -1.1290, -1.2061,
         -0.9378, -0.9761, -0.9734, -1.0249, -1.0257, -0.9283, -0.8040, -0.7299,
         -0.6089, -0.5065, -0.4132, -0.3496, -0.2621, -0.1874, -0.1067, -0.0161,
          0.0455,  0.1743,  0.2912,  0.4558,  0.5812,  0.7106,  0.9250,  1.1623,
          1.4036,  1.8113,  2.2725,  3.0600,  5.4311]], device='cuda:0',
       grad_fn=<AddmmBackward0>)


In [52]:
res = res.cpu()
res = res.detach().numpy()

In [53]:
max_idx = numpy.argmax(res[0])

In [54]:
max_idx

36

In [55]:
res[0]

array([-1.1259978 , -0.7210011 , -0.8748562 , -1.1750541 , -1.1001052 ,
       -1.032618  , -1.1290306 , -1.2061433 , -0.9377701 , -0.97611266,
       -0.97338116, -1.0249169 , -1.0256859 , -0.9283197 , -0.80404633,
       -0.72994834, -0.6088709 , -0.5065379 , -0.41321778, -0.3495546 ,
       -0.26209524, -0.18744344, -0.10667558, -0.01609229,  0.04554788,
        0.17425177,  0.29118493,  0.45575923,  0.5811957 ,  0.71061337,
        0.92497635,  1.1622584 ,  1.4035677 ,  1.8112895 ,  2.272478  ,
        3.0600193 ,  5.4310627 ], dtype=float32)

In [ ]:
class = ['0']